### Autores:  
Blanco García, Gabriel: gabriel.blanco@cunef.edu  
Ferrín Meilán, Michelle: michelle.ferrin@cunef.edu

## Colegio Universitario de Estudios Financieros
### Máster en Data Science para Finanzas
Madrid, diciembre de 2020

# Random Forest

Estos modelos surgen como una alternativa a los árboles de decisión, y el problema que tienen cuando se usan de manera individual. Al utilizar un árbol de decisión, los resultados no son consistentes, y pueden variar en función de cambios en la 
composición de los conjuntos de entrenamiento y validación.  

El Random Forest plantea como solución utilizar muchos árboles, y que la decisión final se tome mediante un sistema de voto. Este sistema se puede modificar, para ponderar el peso de los votos de los árboles en función de, por ejemplo, su precisión.

El Random Forest es bagging, lo que quiere decir que los algoritmos se procesan en paralelo. Cada árbol realiza su partición en conjunto de train y test, y se ejecuta. No hay ningún tipo de regla que influya en qué observaciones puede usar un algortimo, son independeientes de las que haya utilizado otro arbol 

Procedemos a la lecutra de los datos limpios para entrenar el modelo. 

In [1]:
# Manipulacion
import pandas as pd
import numpy as np

# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# El modelo
from sklearn.ensemble import RandomForestClassifier

# Evaluación preliminar
from sklearn import metrics 

# Cargar pipes
import pickle 

# Guardar los modelos
%run ../src/operar_modelos.ipynb

# Omitir warnings
import warnings
warnings.filterwarnings('ignore')

Leemos los datos limpios 

In [9]:
X_train = pd.read_csv('../data/train/X_train.csv', engine='python')
y_train = pd.read_csv('../data/train/y_train.csv', engine='python')

X_test = pd.read_csv('../data/test/X_test.csv', engine='python')
y_test = pd.read_csv('../data/test/y_test.csv', engine='python')

Cargamos los pipelines

In [10]:
preprocesador = cargar_pipelines()

Construimos el modelo, que consta del preprocesado y del casificador Random Forest. No empleamos validación cruzada porque el tiempo de cómputo se extiende demasiado, pero sería la mejor manera de hacerlo

In [11]:
# Levantamos el objeto (no es el entrenamiento)
random_forest = Pipeline(steps=[
    
    # Primer paso: lo llamamos preprocesador y aplicamos el preprocesador generado arriba (con los transformadores dentro)
    ('preprocesador', preprocesador), 
    
    # Segundo paso: lo llamamos clasificador, aplicamos la función RandomForestClassifier()
    ('clasificador', RandomForestClassifier(random_state = 1234, # primero sin parámetros 
                                            n_jobs = 3))]) # 3 procesadores del ordenador para ejecutarlo

Entrenamos el Random Forest

In [12]:
random_forest.fit(X_train, y_train)

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('numericas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('escalador',
                                                                   StandardScaler())]),
                                                  Index(['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'annual_inc_joint',
       'dti_joint', 'mort_acc'],
      dtype='object')),
                                                 ('categoricas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(fill_value='perdido',
                                                                                 strategy='constant')),
                     

In [14]:
random_forest.score(X_train, y_train) # 99.99%

0.999985575917552

In [13]:
random_forest.score(X_test, y_test) # 77.85%, ejemplo claro de overfitting. El modelo no generaliza bien

0.7785566698038706

Guardamos el modelo

In [20]:
guardar_modelo(random_forest, '../models/trained_models/Random_Forest.sav')